In [1]:
import graphlab as gl

In [2]:
import datetime

In [6]:
gf = gl.SFrame('https://static.turi.com/datasets/churn-prediction/online_retail.csv')

Downloading https://static.turi.com/datasets/churn-prediction/online_retail.csv to /var/tmp/graphlab-akhilpunia/18539/a9620fff-7b95-4f1e-9fb9-c870f04b8ea3.csv

Finished parsing file https://static.turi.com/datasets/churn-prediction/online_retail.csv

Parsing completed. Parsed 100 lines in 1.56865 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,int,str,float,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file https://static.turi.com/datasets/churn-prediction/online_retail.csv

Parsing completed. Parsed 541909 lines in 1.40725 secs.

In [7]:
gf

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEARTT-LIGHT HOLDER ...,6,12/1/10 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COATHANGER ...,8,12/1/10 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOTWATER BOTTLE ...,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITEHEART. ...,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,22752,SET 7 BABUSHKA NESTINGBOXES ...,2,12/1/10 8:26,7.65,17850,United Kingdom
536365,21730,GLASS STAR FROSTEDT-LIGHT HOLDER ...,6,12/1/10 8:26,4.25,17850,United Kingdom
536366,22633,HAND WARMER UNION JACK,6,12/1/10 8:28,1.85,17850,United Kingdom
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/10 8:28,1.85,17850,United Kingdom
536367,84879,ASSORTED COLOUR BIRDORNAMENT ...,32,12/1/10 8:34,1.69,13047,United Kingdom


In [10]:
# Convert InvoiceDate from string to datetime.
import dateutil
from dateutil import parser
gf['InvoiceDate'] = gf['InvoiceDate'].apply(parser.parse)

In [12]:
# Convert SFrame into TimeSeries.
time_series = gl.TimeSeries(gf, 'InvoiceDate')

In [13]:
# Create a train-test split.
train, valid = gl.churn_predictor.random_split(time_series,user_id='CustomerID', fraction=0.9)

# Train a churn prediction model.
model = gl.churn_predictor.create(train, user_id='CustomerID',features = ['Quantity'])

PROGRESS: Grouping observation_data by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.
PROGRESS: No time boundaries specified, computing 10 boundaries from 2010-12-01 08:26:00 to 2011-12-09 12:50:00
PROGRESS: Generating features at time-boundaries.
PROGRESS: --------------------------------------------------
PROGRESS: Features for 2011-01-07 21:34:24
PROGRESS: Features for 2011-02-14 05:12:48
PROGRESS: Features for 2011-03-23 12:51:12
PROGRESS: Features for 2011-04-29 20:29:36
PROGRESS: Features for 2011-06-06 04:08:00
PROGRESS: Features for 2011-07-13 11:46:24
PROGRESS: Features for 2011-08-19 19:24:48
PROGRESS: Features for 2011-09-26 03:03:12
PROGRESS: Features for 2011-11-02 10:41:36
PROGRESS: Training a classifier model.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 21490

Number of classes           : 2

...

| Iteration | Elapsed Time | Training-max_error | Training-rmse |

+-----------+--------------+--------------------+---------------+

| 1         | 0.057875     | 0.352540           | 0.188278      |

+-----------+--------------+--------------------+---------------+

PROGRESS: --------------------------------------------------
PROGRESS: Model training complete: Next steps
PROGRESS: --------------------------------------------------
PROGRESS: (1) Evaluate the model at various timestamps in the past:
PROGRESS:       metrics = model.evaluate(data, time_in_past)
PROGRESS: (2) Make a churn forecast for a timestamp in the future:
PROGRESS:       predictions = model.predict(data, time_in_future)


In [14]:
# Making a churn forecast
predictions = model.predict(time_series)

PROGRESS: Making a churn forecast for the time window:
PROGRESS: --------------------------------------------------
PROGRESS:  Start : 2011-12-09 12:50:00
PROGRESS:  End   : 2012-01-08 12:50:00
PROGRESS: --------------------------------------------------
PROGRESS: Grouping dataset by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.
PROGRESS: Generating features for boundary 2011-12-09 12:50:00.


In [15]:
predictions

CustomerID,probability
None,0.0719232484698
12346,0.790395796299
12347,0.551934242249
12348,0.650139987469
12349,0.599330961704
12350,0.824900865555
12352,0.373537153006
12353,0.879394173622
12354,0.879394173622
12355,0.879394173622


In [16]:
# Evaluating the model
evaluation_time = datetime.datetime(2011, 9, 1)
predictions = model.predict(time_series, evaluation_time)

PROGRESS: Making a churn forecast for the time window:
PROGRESS: --------------------------------------------------
PROGRESS:  Start : 2011-09-01 00:00:00
PROGRESS:  End   : 2011-10-01 00:00:00
PROGRESS: --------------------------------------------------
PROGRESS: Grouping dataset by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.
PROGRESS: Generating features for boundary 2011-09-01 00:00:00.
PROGRESS: Not enough data to make predictions for 1005 user(s). 


In [17]:
# Visualize the results
views = model.views.overview(time_series, evaluation_time)
views.show()

PROGRESS: Making a churn forecast for the time window:
PROGRESS: --------------------------------------------------
PROGRESS:  Start : 2011-09-01 00:00:00
PROGRESS:  End   : 2011-10-01 00:00:00
PROGRESS: --------------------------------------------------
PROGRESS: Grouping dataset by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.
PROGRESS: Generating features for boundary 2011-09-01 00:00:00.
PROGRESS: Not enough data to make predictions for 1005 user(s). 
PROGRESS: Making a churn forecast for the time window:
PROGRESS: --------------------------------------------------
PROGRESS:  Start : 2011-09-01 00:00:00
PROGRESS:  End   : 2011-10-01 00:00:00
PROGRESS: --------------------------------------------------
PROGRESS: Grouping dataset by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.
PROGRESS: Generating features for boundary 2011-09-01 00:00:00.
PROGRESS: Not enough data to make predictions for 1005 user(s). 
PROGRE

In [19]:
import pandas as pd

In [33]:
data = pd.read_csv('./Dataset/train.csv')
test = pd.read_csv('./Dataset/test.csv')

/Users/akhilpunia/anaconda/envs/gl-env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
len(data.columns)

377

In [24]:
cols = data.columns

In [25]:
cols[0]

'UCIC_ID'

In [34]:
UCIC_ID = test[cols[0]]

In [35]:
len(UCIC_ID)

200000

In [53]:
Responders = [1 for i in xrange(len(UCIC_ID)/2)]
for i in xrange(len(UCIC_ID)/2):
    Responders.append(0.5)

In [58]:
len(Responders)

200000

In [55]:
sub = pd.DataFrame({'UCIC_ID':UCIC_ID,'Responders':Responders})

In [57]:
sub.to_csv('output.csv',index=False)

In [59]:
cols

Index([u'UCIC_ID', u'NO_OF_Accs', u'HNW_CATEGORY', u'vintage',
       u'EMAIL_UNSUBSCRIBE', u'OCCUP_ALL_NEW', u'city', u'dependents', u'zip',
       u'FINAL_WORTH_prev1',
       ...
       u'Query_Resolved_PrevQ1', u'Complaint_Logged_PrevQ1',
       u'Complaint_Resolved_PrevQ1', u'NO_OF_CHEQUE_BOUNCE_V1',
       u'Percent_Change_in_Credits', u'Percent_Change_in_FT_Bank',
       u'Percent_Change_in_FT_outside', u'Percent_Change_in_Self_Txn',
       u'Percent_Change_in_Big_Expenses', u'Responders'],
      dtype='object', length=377)